## Import modules

In [1]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

## Data loading

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset'

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\Dataset\\bat_dict.pkl'

In [3]:
bat_dict_add_1_2 = find_directory(back_num = 2, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

In [4]:
# Tensor dimension: (cell, cycle, time, variable)
num_cells = len(bat_sel_dict.keys())
regression_cycles = [45, 65, 85, 100]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    #'''RNN 가변길이 다룰수 있게?? 고민필요'''
    globals()[f'x_tensor_{reg}'] = torch.zeros(num_cells, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1

    globals()[f'x_tensor_{reg}'].size()

num_variables = x_tensor_100.shape[3]
time_length = x_tensor_100.shape[2]
# (cycle_index-1, cycle-1, time-1, variable(Qc, I, V, T, Qd))
# x_tensor[123, 99, 30, :]

torch.Size([124, 45, 120, 5])

torch.Size([124, 65, 120, 5])

torch.Size([124, 85, 120, 5])

torch.Size([124, 100, 120, 5])

In [5]:
y_df_add = find_directory(back_num = 2, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
y_ko_tensor.size()

torch.Size([124])

In [6]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

In [7]:
x_tensor_100.shape

torch.Size([124, 100, 120, 5])

### Dataloading as 2D CNN's method

In [8]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, purpose : str, test_split: float):
        setRandomSeed(random_seed = 100)
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        self.purpose = purpose
        
        self.test_split = test_split
        
        self.train_size, self.val_size, self.test_size = self.get_splits()
        
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
        self.x_train_scaled, self.x_test_scaled, self.y_train_scaled, self.y_test_scaled = train_test_split(self.xdata_scaled, 
                                                                                                            self.ydata_scaled, 
                                                                                                            test_size = self.test_size, 
                                                                                                            shuffle = True, 
                                                                                                            random_state=100)
        
        self.x_train_scaled, self.x_val_scaled, self.y_train_scaled, self.y_val_scaled = train_test_split(self.x_train_scaled,
                                                                                                         self.y_train_scaled,
                                                                                                         test_size = self.val_size, 
                                                                                                         shuffle = True, 
                                                                                                         random_state = 100)
        
        print(self.x_train_scaled.shape, self.x_val_scaled.shape, self.x_test_scaled.shape,
              self.y_train_scaled.shape, self.y_val_scaled.shape, self.y_test_scaled.shape)
        
        self.train = list(zip(self.x_train_scaled, self.y_train_scaled))
        self.val = list(zip(self.x_val_scaled, self.y_val_scaled))
        self.test = list(zip(self.x_test_scaled, self.y_test_scaled))
                
        self.train_dataloader = DataLoader(self.train, batch_size = self.batch_size, shuffle=True)
        self.val_dataloader = DataLoader(self.val, batch_size = self.batch_size, shuffle=True)
        self.test_dataloader = DataLoader(self.test, batch_size = self.batch_size, shuffle=True)
            
    # split the datasets into training and testing
    def get_splits(self):
        # test size
        test_size = int(self.test_split*len(self.xdata))
        # val size
        val_size = int(0.2*(len(self.xdata) - test_size))
        # train_size
        train_size = len(self.xdata) - val_size - test_size
        return train_size, val_size, test_size
    
    # Returns dataloaders for training and validation datasets
    def train_data(self):
        return self.train_dataloader, self.val_dataloader
    
    # Returns dataloader for test datasets
    def test_data(self):
        return self.test_dataloader
    
    def scaler(self, purpose):
#         if purpose != 'classification':
#             return self.xscaler, self.yscaler
#         else:
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]


In [9]:
# reg_dataset = dataPrep1D(x_tensor, y_ko_tensor, batch_size = 16, scaler = MinMaxScaler, purpose = 'regression', test_split = 0.2)
# reg_train_dataloader, reg_val_dataloader = reg_dataset.train_data()
# reg_test_dataloader = reg_dataset.test_data()
# reg_xscaler = reg_dataset.scaler('regression')

regression_cycles = [45, 65, 85, 100]
# regression_cycles = [100]


# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for reg in regression_cycles:
    globals()[f'reg_dataset_{reg}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, batch_size = 4, scaler = MinMaxScaler, purpose =  'regression', test_split = 0.2)
    globals()[f'reg_train_dataloader_{reg}'], globals()[f'reg_val_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].train_data()
    globals()[f'reg_test_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].test_data()
    globals()[f'reg_xscaler_{reg}'] = globals()[f'reg_dataset_{reg}'].scaler('regression')

(80, 5, 120, 45) (20, 5, 120, 45) (24, 5, 120, 45) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 65) (20, 5, 120, 65) (24, 5, 120, 65) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 85) (20, 5, 120, 85) (24, 5, 120, 85) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 100) (20, 5, 120, 100) (24, 5, 120, 100) torch.Size([80]) torch.Size([20]) torch.Size([24])


In [10]:
# train_features, train_labels = next(iter(cla_train_dataloader))
# print(f"Classification case_Inputs batch shape(Batch size, num_vars, timestep, cycles): {train_features.size()}")
# print(f"Classification case_Outputs batch shape(Batch size): {train_labels.size()}")

reg = 100
train_features, train_labels = next(iter(globals()[f'reg_train_dataloader_{reg}']))
print(f'reg = {reg}')
print(f"Regression case_Inputs batch shape(Batch size, num_vars, timestep, cycles): {train_features.size()}")
print(f"Regression case_Outputs batch shape(Batch size): {train_labels.size()}")

train_features.float().type()

reg = 100
Regression case_Inputs batch shape(Batch size, num_vars, timestep, cycles): torch.Size([4, 5, 120, 100])
Regression case_Outputs batch shape(Batch size): torch.Size([4])


'torch.FloatTensor'

## Define Model Class 

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

In [13]:
class RNN_TA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, fil2, pool2, npool2, fsize2, psize2, mids, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_TA_1DCNN, self).__init__()
        setRandomSeed()
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
        self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
        ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
        ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
        ct_vec = torch.sum(ta_outs, -1)
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ta.squeeze()

In [14]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

# Early stopping class
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 1
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, epoch, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.best_model = self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'At epoch {self.best_epoch}: Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        return model

In [15]:
class train:
    def __init__(self, model: nn.Module, train_dataloader, val_dataloader, test_dataloader, epoch: int, learning_rate=0.01, patience = 5, verbose = False):
        super().__init__()
        #   Reprocudtion
        setRandomSeed(100)
        random_seed = 100
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(random_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        os.environ['PYTHONHASHSEED'] = str(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)

        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.test_dataloader = test_dataloader
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.early_stopping = EarlyStopping(patience = patience, verbose = verbose)

        self.criterion = nn.MSELoss()
        self.name = ['train_rmse', 'val_rmse', 'test_rmse']
        
    def EvalModel(self):
        EvalPredictions, EvalActuals = list(), list()
        TestPredictions, TestActuals = list(), list()
        self.model.eval()
        with torch.no_grad():
            for n, (inputs, targets) in enumerate(self.val_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                EvalPredictions.append(yhat)
                EvalActuals.append(actual)

            EvalPredictions, EvalActuals = vstack(EvalPredictions), vstack(EvalActuals)
            val_rmse = mean_squared_error(EvalActuals, EvalPredictions, squared = False)
            
            for n, (inputs, targets) in enumerate(self.test_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                TestPredictions.append(yhat)
                TestActuals.append(actual)

            TestPredictions, TestActuals = vstack(TestPredictions), vstack(TestActuals)
            test_rmse = mean_squared_error(TestActuals, TestPredictions, squared = False)
            
            return val_rmse, test_rmse

    def TrainModel(self):
        loss_history = []
        self.model.train()
        for i in range(self.epoch):
            predictions, actuals = list(), list()
            for n, (inputs, targets) in enumerate(self.train_dataloader):
                self.model.train()
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                self.optimizer.zero_grad()

                yhat, ta = self.model(inputs)
                    
                loss = self.criterion(yhat, targets)
                loss.backward()
                self.optimizer.step()
                
                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')
                    
                yhat = yhat.detach().numpy()
                predictions.append(yhat)
                
                targets = targets.numpy()
                actuals.append(targets)

            # evaluation of entire train data
            predictions, actuals = vstack(predictions), vstack(actuals)
            
            train_rmse = mean_squared_error(actuals, predictions, squared = False)

            # evaluation on validation data
            val_rmse, test_rmse = self.EvalModel()
            early_factor = val_rmse
            
            # training loss history
            loss_history.append([train_rmse, val_rmse, test_rmse])

            self.early_stopping(i+1, early_factor, self.model)
            
            if self.early_stopping.early_stop:
                print("Early stopping at best epoch: ", self.early_stopping.best_epoch)
                self.best_loss = loss_history[self.early_stopping.best_epoch-1]
                loss_history = loss_history[:self.early_stopping.best_epoch]
                
                return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta
            
        print(f"Ends at final epoch {self.epoch}")
        print(f"Best epoch: {self.early_stopping.best_epoch}")
        self.best_loss = loss_history[self.early_stopping.best_epoch-1]
        loss_history = loss_history[:self.early_stopping.best_epoch]
        
        return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta
        
    def predict(self, data: torch.Tensor):
        return self.model(data)
    
    def train_val_test_plot_rmse(self, num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr):
        dataloaders = [self.train_dataloader, self.val_dataloader, self.test_dataloader]

        with torch.no_grad():
            model.eval()

            model_rmse = list()
            dset_index = 0
            for dataloader in dataloaders:
                if dset_index ==0:
                    dset = 'train'
                elif dset_index == 1:
                    dset = 'val'
                else:
                    dset = 'test'

                figadd = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.jpg')

                predictions, actuals = list(), list()
                for i, (test_input, test_target) in enumerate(dataloader):
                    test_input = test_input.float().to(self.device)
                    test_target = test_target.float().to(self.device)
                    test_yhat, test_ta = model(test_input)

                    if self.device != 'cpu':
                        test_yhat = test_yhat.to('cpu')
                        test_target = test_target.to('cpu')

                    test_yhat = test_yhat.detach().tolist()
                    predictions += test_yhat
                    test_target = test_target.tolist()
                    actuals += test_target

                pred_acc, targ_acc = np.array(predictions), np.array(actuals)

                rmse = mean_squared_error(actuals, predictions, squared = False)
                model_rmse.append(rmse)

                plt.figure(figsize=[10,4])
                plt.rcParams["font.size"] = "12"
                plt.plot(pred_acc, label='Prediction')
                plt.plot(range(len(targ_acc)), targ_acc, label = 'Actual')
                plt.xticks(range(0, len(targ_acc) + 1, int(len(targ_acc)/5)))
                plt.title(f"{rnn1}_hid_{hid1}_nfil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}: RMSE={rmse}")
                plt.xlabel('cell')
                plt.ylabel('Lifespan(cycle)')
                plt.legend()
                plt.savefig(figadd)
                plt.close()

                dset_index += 1

        return model_rmse

## Implementation

In [16]:
def history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add

In [17]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                start = time.time()
                print(f"rnn = {rnn1}, bi = {bi1}, hid = {hid1}, num_fil = {fil2}, lr = {lr}")
                
                # Construct CNN
                model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, hid1, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                # Train
                model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                best_model, best_loss, history, ta = model_train.TrainModel()

                print('best_loss = '+ str(best_loss))

                history_add, state_dict_add, ta_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                            rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr)
                
                # saving best_epoch, loss history, ta score
                save_data(history, history_add)
                save_data(best_model.state_dict(), state_dict_add)
                save_data(ta, ta_add)

                print("time: ", time.time()-start)
                print("\n\n")

epoch = 1000, patience = 10
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  13
best_loss = [577.79425, 542.3338, 522.103]
time:  4.199462413787842



rnn = LSTM, bi = False, hid = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  33
best_loss = [255.0834, 172.3768, 120.02615]
time:  4.1010048389434814



rnn = LSTM, bi = False, hid = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  33
best_loss = [153.60907, 98.18982, 72.558174]
time:  4.0960047245025635



rnn = LSTM, bi = False, hid = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  161
best_loss = [254.86148, 179.30984, 130.96541]
time:  16.36801838874817



rnn = LSTM, bi = False, hid = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  20
best_loss = [261.13672, 170.90521, 115.49716]
time:  2.8750030994415283



rnn = LSTM, bi = False, hid = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  29


Early stopping at best epoch:  162
best_loss = [226.64032, 155.59, 117.198906]
time:  17.738020181655884



rnn = LSTM, bi = False, hid = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  20
best_loss = [243.07227, 150.54135, 102.28107]
time:  3.1160037517547607



rnn = LSTM, bi = False, hid = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  22
best_loss = [142.7456, 110.95662, 78.38164]
time:  3.3040034770965576



rnn = LSTM, bi = False, hid = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [572.06226, 543.5937, 528.1134]
time:  1.548001766204834



rnn = LSTM, bi = False, hid = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  5
best_loss = [571.94165, 543.5554, 528.074]
time:  1.554001808166504



rnn = LSTM, bi = False, hid = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  99
best_loss = [286.43658, 214.47859, 159.54147]
time:  11.24201250076294



Pooling: 2 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = 

Early stopping at best epoch:  20
best_loss = [244.08383, 167.05266, 124.345085]
time:  3.0610039234161377



rnn = LSTM, bi = False, hid = 7, num_fil = 7, lr = 0.01
Early stopping at best epoch:  20
best_loss = [151.09386, 117.39169, 74.664055]
time:  3.0570027828216553



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid = 7, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  161
best_loss = [246.711, 180.7821, 135.81708]
time:  18.378736972808838



rnn = LSTM, bi = False, hid = 7, num_fil = 3, lr = 0.001
Early stopping at best epoch:  20
best_loss = [242.108, 152.26465, 120.27904]
time:  3.232003927230835



rnn = LSTM, bi = False, hid = 7, num_fil = 3, lr = 0.01
Early stopping at best epoch:  11
best_loss = [181.60162, 126.95009, 70.60702]
time:  2.2640023231506348



rnn = LSTM, bi = False, hid = 7, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  185
best_loss = [243.05731, 177.62448, 124.22484]
time:  21.743024587631226



rnn = L

Early stopping at best epoch:  15
best_loss = [191.43568, 131.9145, 84.75076]
time:  2.3460023403167725



rnn = GRU, bi = False, hid = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [570.82916, 542.20526, 526.7062]
time:  1.4120018482208252



rnn = GRU, bi = False, hid = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  36
best_loss = [249.9412, 164.13138, 109.39697]
time:  4.332005023956299



rnn = GRU, bi = False, hid = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  14
best_loss = [153.8451, 124.25212, 73.27257]
time:  2.2610023021698



rnn = GRU, bi = False, hid = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  150
best_loss = [238.84973, 182.30591, 123.34787]
time:  15.095016956329346



rnn = GRU, bi = False, hid = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  20
best_loss = [255.08778, 164.70724, 112.60118]
time:  2.843003273010254



rnn = GRU, bi = False, hid = 5, num_fil = 7, lr = 0.01
Early stopping at bes

Early stopping at best epoch:  99
best_loss = [259.57532, 187.20213, 125.008095]
time:  10.980515956878662



rnn = GRU, bi = False, hid = 7, num_fil = 7, lr = 0.001
Early stopping at best epoch:  20
best_loss = [236.86613, 148.68382, 106.190735]
time:  3.02500319480896



rnn = GRU, bi = False, hid = 7, num_fil = 7, lr = 0.01
Early stopping at best epoch:  14
best_loss = [144.7833, 143.74396, 84.492584]
time:  2.4230029582977295



Pooling: 2 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = GRU, bi = False, hid = 7, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [571.99835, 543.5445, 528.05444]
time:  1.5140016078948975



rnn = GRU, bi = False, hid = 7, num_fil = 3, lr = 0.001
Early stopping at best epoch:  36
best_loss = [214.92532, 154.8595, 112.554955]
time:  4.665005207061768



rnn = GRU, bi = False, hid = 7, num_fil = 3, lr = 0.01
Early stopping at best epoch:  12
best_loss = [207.15843, 147.96072, 92.5839]
time:  2.2230024337768555



rnn = GRU, bi =

...

Early stopping at best epoch:  411
best_loss = [236.83545, 147.52705, 115.37909]
time:  91.5479166507721



rnn = GRU, bi = False, hid = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  888
best_loss = [98.76004, 121.43146, 82.43738]
time:  139.41434288024902



rnn = GRU, bi = False, hid = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  91
best_loss = [113.90805, 96.62075, 77.01498]
time:  59.32106685638428



rnn = GRU, bi = False, hid = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1649
best_loss = [136.09895, 86.57973, 68.76687]
time:  213.29323983192444



rnn = GRU, bi = False, hid = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  281
best_loss = [125.018845, 106.697, 62.207775]
time:  77.50886154174805



rnn = GRU, bi = False, hid = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  58
best_loss = [126.4837, 113.579994, 72.67135]
time:  55.35600209236145



rnn = GRU, bi = False, hid = 3, num_fil = 7, lr = 0.0001
Early stopping at 

Early stopping at best epoch:  1241
best_loss = [127.14066, 111.11957, 58.509773]
time:  190.11424589157104



rnn = GRU, bi = False, hid = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  61
best_loss = [165.33289, 118.51866, 75.717384]
time:  61.254090785980225



rnn = GRU, bi = False, hid = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  299
best_loss = [20.176214, 121.97065, 81.23934]
time:  87.34589576721191



rnn = GRU, bi = False, hid = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1110
best_loss = [134.95015, 104.14467, 79.2012]
time:  179.41659927368164



rnn = GRU, bi = False, hid = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  61
best_loss = [213.93394, 141.89612, 104.40383]
time:  62.447530031204224



rnn = GRU, bi = False, hid = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  440
best_loss = [36.751804, 113.50932, 124.43912]
time:  104.71609687805176



Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = G

Early stopping at best epoch:  347
best_loss = [115.82152, 119.233665, 107.75133]
time:  79.35516834259033



rnn = RNN, bi = False, hid = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  29
best_loss = [149.43399, 121.96037, 70.82995]
time:  49.63343596458435



rnn = RNN, bi = False, hid = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1212
best_loss = [169.12814, 111.10942, 86.31244]
time:  161.21412181854248



rnn = RNN, bi = False, hid = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  533
best_loss = [106.872154, 92.716034, 83.849174]
time:  97.42377662658691



rnn = RNN, bi = False, hid = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  37
best_loss = [141.698, 101.03897, 80.18065]
time:  50.58889961242676



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = RNN, bi = False, hid = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1212
best_loss = [170.9054, 110.60368, 76.52722]
time:  170.16712737083435



rnn = RNN, 

Early stopping at best epoch:  173
best_loss = [124.475334, 91.93295, 93.595474]
time:  66.43214678764343



rnn = RNN, bi = False, hid = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  23
best_loss = [128.02687, 129.00154, 92.8203]
time:  51.69453191757202



Pooling: 2 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = RNN, bi = False, hid = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  2339
best_loss = [132.02393, 73.83093, 82.468414]
time:  279.6043698787689



rnn = RNN, bi = False, hid = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  292
best_loss = [121.69699, 86.60683, 86.96309]
time:  78.07653450965881



rnn = RNN, bi = False, hid = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  945
best_loss = [38.248238, 94.64726, 99.04154]
time:  142.05998063087463



rnn = RNN, bi = False, hid = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  411
best_loss = [234.45941, 138.55524, 113.05509]
time:  89.05123257637024



rnn = RNN, bi 

Ends at final epoch 3000
Best epoch: 2780
best_loss = [533.7693, 476.78903, 469.68848]
time:  319.1859395503998



rnn = RNN, bi = False, hid = 7, num_fil = 3, lr = 0.01
Ends at final epoch 3000
Best epoch: 2955
best_loss = [288.5016, 200.04362, 168.62146]
time:  318.505051612854



rnn = RNN, bi = False, hid = 7, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1394
best_loss = [134.48709, 112.592285, 60.478195]
time:  208.50063371658325



rnn = RNN, bi = False, hid = 7, num_fil = 5, lr = 0.001
Early stopping at best epoch:  921
best_loss = [42.234406, 69.944336, 103.59474]
time:  156.37430953979492



rnn = RNN, bi = False, hid = 7, num_fil = 5, lr = 0.01
Early stopping at best epoch:  358
best_loss = [43.730446, 118.2232, 97.108246]
time:  94.45598363876343



rnn = RNN, bi = False, hid = 7, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  411
best_loss = [168.5676, 112.79857, 76.55082]
time:  102.14549541473389



rnn = RNN, bi = False, hid = 7, num_fil = 7, lr = 0.0

In [18]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_1DCNN'
rnns = ['BiLSTM', 'BiGRU', 'BiRNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                start = time.time()
                print(f"rnn = {rnn1}, bi = {bi1}, hid = {hid1}, num_fil = {fil2}, lr = {lr}")
                
                # Construct CNN
                model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, hid1, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                # Train
                model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                best_model, best_loss, history, ta = model_train.TrainModel()

                print('best_loss = '+ str(best_loss))

                history_add, state_dict_add, ta_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                            rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr)
                
                # saving best_epoch, loss history, ta score
                save_data(history, history_add)
                save_data(best_model.state_dict(), state_dict_add)
                save_data(ta, ta_add)

                print("time: ", time.time()-start)
                print("\n\n")

epoch = 1000, patience = 10
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = BiLSTM, bi = True, hid = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [571.9298, 543.5017, 527.94995]
time:  1.5270535945892334



rnn = BiLSTM, bi = True, hid = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  36
best_loss = [243.22621, 154.79492, 121.38132]
time:  4.684005260467529



rnn = BiLSTM, bi = True, hid = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  11
best_loss = [233.54474, 149.29227, 100.44436]
time:  2.134002447128296



rnn = BiLSTM, bi = True, hid = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [572.46826, 544.152, 528.7074]
time:  1.5294559001922607



rnn = BiLSTM, bi = True, hid = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  5
best_loss = [572.3739, 544.0374, 528.56134]
time:  1.5311083793640137



rnn = BiLSTM, bi = True, hid = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  11

Early stopping at best epoch:  15
best_loss = [188.94289, 152.01636, 123.59895]
time:  2.7204525470733643



rnn = BiLSTM, bi = True, hid = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [572.12836, 543.7517, 528.26404]
time:  1.602003574371338



rnn = BiLSTM, bi = True, hid = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  36
best_loss = [238.7467, 162.05591, 120.19766]
time:  4.9146409034729



rnn = BiLSTM, bi = True, hid = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  20
best_loss = [187.83899, 144.14365, 90.39256]
time:  3.200021266937256



rnn = BiLSTM, bi = True, hid = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [571.4835, 542.9856, 527.49963]
time:  1.6200039386749268



rnn = BiLSTM, bi = True, hid = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  20
best_loss = [257.77362, 169.43253, 125.270004]
time:  3.2512829303741455



rnn = BiLSTM, bi = True, hid = 5, num_fil = 7, lr = 0.01
Early sto

Early stopping at best epoch:  20
best_loss = [169.79071, 141.73708, 76.8322]
time:  3.2353029251098633



rnn = BiLSTM, bi = True, hid = 7, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  162
best_loss = [230.91682, 159.88301, 115.87243]
time:  18.40059781074524



rnn = BiLSTM, bi = True, hid = 7, num_fil = 7, lr = 0.001
Early stopping at best epoch:  20
best_loss = [247.8052, 153.98874, 94.13368]
time:  3.2470052242279053



rnn = BiLSTM, bi = True, hid = 7, num_fil = 7, lr = 0.01
Early stopping at best epoch:  18
best_loss = [135.51057, 114.29214, 73.39927]
time:  3.0282046794891357



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = BiLSTM, bi = True, hid = 7, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  31
best_loss = [553.28284, 510.44165, 498.76364]
time:  4.632359981536865



rnn = BiLSTM, bi = True, hid = 7, num_fil = 3, lr = 0.001
Early stopping at best epoch:  48
best_loss = [224.4109, 124.31471, 102.19456]
time:  6.556490659713745



rnn =

Early stopping at best epoch:  161
best_loss = [241.99847, 176.10855, 113.263504]
time:  20.021687030792236



rnn = BiGRU, bi = True, hid = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  36
best_loss = [217.92497, 140.03119, 105.516754]
time:  5.398256778717041



rnn = BiGRU, bi = True, hid = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  20
best_loss = [139.16321, 133.73724, 65.09865]
time:  3.516191244125366



Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = BiGRU, bi = True, hid = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [571.9802, 543.5215, 528.0681]
time:  1.5294625759124756



rnn = BiGRU, bi = True, hid = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  48
best_loss = [239.22511, 165.27383, 103.19514]
time:  5.9146459102630615



rnn = BiGRU, bi = True, hid = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  37
best_loss = [129.01083, 104.484245, 68.28965]
time:  4.783780336380005



rnn = BiG

Early stopping at best epoch:  5
best_loss = [572.49164, 544.16797, 528.6882]
time:  1.627892255783081



rnn = BiGRU, bi = True, hid = 7, num_fil = 3, lr = 0.001
Early stopping at best epoch:  5
best_loss = [572.4024, 544.0675, 528.5918]
time:  1.6269361972808838



rnn = BiGRU, bi = True, hid = 7, num_fil = 3, lr = 0.01
Early stopping at best epoch:  5
best_loss = [571.71204, 543.246, 527.76196]
time:  1.6268165111541748



rnn = BiGRU, bi = True, hid = 7, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  161
best_loss = [269.0252, 196.96906, 146.72856]
time:  18.73726487159729



rnn = BiGRU, bi = True, hid = 7, num_fil = 5, lr = 0.001
Early stopping at best epoch:  36
best_loss = [223.40167, 156.16623, 104.41607]
time:  5.0569539070129395



rnn = BiGRU, bi = True, hid = 7, num_fil = 5, lr = 0.01
Early stopping at best epoch:  20
best_loss = [158.81387, 140.22792, 85.75913]
time:  3.2906792163848877



rnn = BiGRU, bi = True, hid = 7, num_fil = 7, lr = 0.0001
Early stopping a

...

Early stopping at best epoch:  1241
best_loss = [155.9797, 112.42903, 53.88008]
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_3000_500\RNN_TA_1DCNN\BiGRU_1D CNN\train_history created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_3000_500\RNN_TA_1DCNN\BiGRU_1D CNN\model created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_3000_500\RNN_TA_1DCNN\BiGRU_1D CNN\ta created
time:  175.4317650794983



rnn = BiGRU, bi = True, hid = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  185
best_loss = [112.09509, 106.274796, 92.038574]
time:  68.99888920783997



rnn = BiGRU, bi = True, hid = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  36
best_loss = [147.27808, 89.77911, 94.40955]
time:  53.84193658828735



rnn = BiGRU, bi = True, hid = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1212
best_loss = [150.80182, 120.85564, 

Early stopping at best epoch:  89
best_loss = [153.84073, 136.40007, 63.63852]
time:  63.601484537124634



rnn = BiGRU, bi = True, hid = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  15
best_loss = [149.9918, 128.23975, 84.54228]
time:  55.585015058517456



rnn = BiGRU, bi = True, hid = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1302
best_loss = [111.49232, 109.53154, 89.708206]
time:  196.41994905471802



rnn = BiGRU, bi = True, hid = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  411
best_loss = [74.31133, 109.293274, 85.31828]
time:  99.21498394012451



rnn = BiGRU, bi = True, hid = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  19
best_loss = [147.3335, 130.72202, 73.69871]
time:  56.41828107833862



rnn = BiGRU, bi = True, hid = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1142
best_loss = [167.764, 111.002396, 67.92098]
time:  176.29542446136475



rnn = BiGRU, bi = True, hid = 5, num_fil = 7, lr = 0.001
Early sto

Early stopping at best epoch:  61
best_loss = [207.5603, 138.10661, 94.415344]
time:  60.64917254447937



rnn = BiGRU, bi = True, hid = 7, num_fil = 5, lr = 0.01
Early stopping at best epoch:  912
best_loss = [48.568417, 94.98917, 106.5607]
time:  152.68010663986206



rnn = BiGRU, bi = True, hid = 7, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  411
best_loss = [217.33937, 123.31139, 105.049255]
time:  98.03892064094543



rnn = BiGRU, bi = True, hid = 7, num_fil = 7, lr = 0.001
Early stopping at best epoch:  111
best_loss = [139.82748, 137.7775, 63.280334]
time:  65.69108438491821



rnn = BiGRU, bi = True, hid = 7, num_fil = 7, lr = 0.01
Early stopping at best epoch:  688
best_loss = [25.370998, 147.74777, 75.72179]
time:  127.57742094993591



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = BiGRU, bi = True, hid = 7, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  411
best_loss = [212.10104, 133.57513, 105.11634]
time:  103.01229095458984



rnn =

Ends at final epoch 3000
Best epoch: 2955
best_loss = [288.55615, 200.0668, 168.7055]
time:  337.52372336387634



rnn = BiRNN, bi = True, hid = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1302
best_loss = [112.978714, 104.58271, 65.96252]
time:  206.88137459754944



rnn = BiRNN, bi = True, hid = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  95
best_loss = [144.4718, 124.34165, 80.4646]
time:  68.39109802246094



rnn = BiRNN, bi = True, hid = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  20
best_loss = [149.09714, 119.967804, 66.498375]
time:  59.635822057724



Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = BiRNN, bi = True, hid = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1560
best_loss = [125.55881, 130.33922, 90.27365]
time:  204.1511995792389



rnn = BiRNN, bi = True, hid = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  1382
best_loss = [35.883358, 94.484985, 83.73042]
time:  186.61148238182068




Early stopping at best epoch:  248
best_loss = [38.934013, 106.78558, 139.0558]
time:  75.19773364067078



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = BiRNN, bi = True, hid = 7, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1212
best_loss = [166.8771, 110.29219, 90.99425]
time:  181.16523456573486



rnn = BiRNN, bi = True, hid = 7, num_fil = 3, lr = 0.001
Early stopping at best epoch:  278
best_loss = [119.300354, 104.804886, 79.50779]
time:  82.17520570755005



rnn = BiRNN, bi = True, hid = 7, num_fil = 3, lr = 0.01
Early stopping at best epoch:  61
best_loss = [126.89555, 85.11631, 96.64877]
time:  59.32647490501404



rnn = BiRNN, bi = True, hid = 7, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1986
best_loss = [574.573, 527.36816, 522.53296]
time:  265.7805778980255



rnn = BiRNN, bi = True, hid = 7, num_fil = 5, lr = 0.001
Early stopping at best epoch:  311
best_loss = [68.98326, 116.42122, 101.26899]
time:  86.77121543884277



rnn = B

In [ ]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

for [n_ep, patience] in ep_pats:
    for rnn1 in rnns:
        df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
        bi1 = False
        if rnn1.startswith('Bi'):
            bi1 = True
        for hid1 in hids:
            for pool2, npool2 in itertools.product(pools, npools):
                print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers')
                for fil2, lr in itertools.product(num_fils, lrs):
                    history_add, _, _ = history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr)
                    history = load_data(history_add)
                    print(history.iloc[-1, :])

                    df.loc[f'{rnn1}_{hid1}_{fil2}_{pool2}_{npool2}_{fsize2}_{lr}'] = [history.iloc[-1, 0], history.iloc[-1, 1], history.iloc[-1, 2]]

        df
        df_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname], 
                        filename = f'{modelname}_{rnn1}.csv')
        df.to_csv(df_add)
                                                                         

## Plot train/test results

In [20]:
n_fils = [4, 8, 16]
fsizes = [3]
lrs = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3]
modelname = 'RNN_TA_1DCNN'
modelname2s = ['RNN', 'LSTM', 'GRU']

for n_cy in regression_cycles:
    for n_fil in n_fils:
        for fsize in fsizes:
            for lr in lrs:
                for modelname2 in modelname2s:
                    save_add = f'split_1_{int(max_time/60)}min_padding_{padding_layer}_layers_nonpadding_{nonpadding_layer}_layers_n_fil_{n_fil}_f_size_{fsize}_lr_1_{int(1/lr)}'
                    
                    epoch_add = find_directory(foldername = [f'{n_cy} cycles', modelname, modelname2, 'best_epoch', f'{save_add}.wb'])
                    history_add = find_directory(foldername = [f'{n_cy} cycles', modelname, modelname2, 'train_history', f'{save_add}.pkl'])
                
                    history = load_data(history_add)
                    best_epoch = load_data(epoch_add)
                
                    fig, ax1 = plt.subplots(figsize = (10,4));
                    plt.rcParams["font.size"] = "12";
                    line1 = ax1.plot(np.sqrt(history['mse_loss']), 'y', label = 'train RMSE');
                    line2 = ax1.plot(np.sqrt(history['valmse_loss']), 'b', label = 'val RMSE');
                    ax1.set_xlabel('Epoch');
                    ax1.set_ylabel('RMSE');
                    ax1.set_title(f'{modelname}, {modelname2}, n_fil: {n_fil}, fsize: {fsize}, lr: {lr}, best_epoch: {best_epoch}')

                    ax2 = ax1.twinx();
                    plt.rcParams["font.size"] = "12";
                    line3 = ax2.plot(history['mape_loss'], 'r', label = 'train MAPE');
                    line4 = ax2.plot(history['valmape_loss'], 'g', label = 'val MAPE');
                    ax2.set_ylabel('MAPE');
                    ax2.set_ylim(0, 1);

                    lines = line1 + line2 + line3 + line4;
                    labels = [l.get_label() for l in lines];
                    ax1.legend(lines, labels, loc = 'best');

                figadd = find_directory(foldername = [f'{n_cy} cycles', modelname, modelname2, 'train_val_errors', f'split_1_{int(max_time/60)}min_{n_fil}_{fsize}_{lr}_{best_epoch}.jpg'])
                plt.savefig(figadd);

NameError: name 'padding_layer' is not defined

In [ ]:
regression_cycles = [45, 65, 85, 100]
n_fils = [4, 8, 16]
# purpose = purposes[1]
psize = 2
# n_fils = [4]
fsizes = [3]
lrs = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]
hidden_size  =3
dsets = ['train', 'val', 'test']

modelname = 'RNN_TA_1DCNN'
modelname2s = ['RNN', 'LSTM', 'GRU']

for n_cy in regression_cycles:
    trdl = globals()[f'reg_train_dataloader_{n_cy}']
    vdl = globals()[f'reg_val_dataloader_{n_cy}']
    tedl = globals()[f'reg_test_dataloader_{n_cy}']
    for modelname2 in modelname2s:
        print(modelname2, ':', n_cy, ' cycles')
        df = pd.DataFrame(columns = ['train mape', 'train rmse', 'val mape', 'val rmse', 'test mape', 'test rmse'])
        for n_fil in n_fils:
            for fsize in fsizes:
                for lr in lrs:
#                     purpose = purposes[1]
                    with torch.no_grad():
                        if torch.cuda.is_available():
                            device = 'cuda'
                        else:
                            device = 'cpu'
                            
                        model = globals()[modelname](rnn_num_layer=1, model = modelname2, input_size = n_var, bidirectional = False, hidden_size = hidden_size, num_filter = n_fil,
                              padding_layer = padding_layer, nonpadding_layer = nonpadding_layer, filter_size = fsize, pooling_size = psize,
                              num_time = max_time, num_cycles = n_cy).to(device)
                        model_add = f'split_1_{int(max_time/60)}min_padding_{padding_layer}_layers_nonpadding_{nonpadding_layer}_layers_n_fil_{n_fil}_f_size_{fsize}_lr_1_{int(1/lr)}_state_dict.pth'
                        state_dict = load_data(os.getcwd() + f'\\{n_cy} cycles\\{modelname}\\{modelname2}\\model\\'+model_add)
                        model.load_state_dict(state_dict)
                        model.eval()

                        model_mape, model_rmse = list(), list()


                        for dset in dsets:
                            if dset == 'train':
                                dataloader = trdl
                            elif dset == 'val':
                                dataloader = vdl
                            else:
                                dataloader = tedl

                            predictions, actuals = list(), list()

                            for i, (test_input, test_target) in enumerate(dataloader):
                                test_input = test_input.float().to(device)
                                test_target = test_target.float().to(device)

                                test_yhat, (ta_weight, ta_bias) = model(test_input)

                                if device != 'cpu':
                                    test_yhat = test_yhat.to('cpu')
                                    test_target = test_target.to('cpu')

                                test_yhat = test_yhat.detach().numpy()
                                predictions.append(test_yhat)
                                test_target = test_target.numpy()
                                actuals.append(test_target)

                            pred_acc, targ_acc = np.array(predictions).flatten(), np.array(actuals).flatten()


                            mse = mean_squared_error(actuals, predictions)
                            mape = mean_absolute_percentage_error(actuals, predictions)

                            model_mape.append(mape)
                            model_rmse.append(np.sqrt(mse))

                        df.loc[f'{n_fil}_{fsize}_{lr}'] = [model_mape[0], model_rmse[0], model_mape[1], model_rmse[1], model_mape[2], model_rmse[2]]


        df
        df.to_csv(os.getcwd()+f'\\{n_cy} cycles\\{modelname}\\{modelname2}\\split_1_{int(max_time/60)}min_{modelname}_{modelname2} padding_{padding_layer}_layers_nonpadding_{nonpadding_layer}_layers errors_{n_cy} cycles.csv')